# Leaf Disease Detection
In this notebook, you would need to implement a CNN classifier for leaf disease detection. Your goal is to submit your predictions to the competition! Feel free to use previous case studies, but make sure you understand what the code is doing before using it.

In [ ]:
import os
from os.path import join

from tqdm.notebook import tqdm # progress bar

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.models import Model

import cv2

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split

from IPython.display import SVG


## Load Data

In [ ]:
IMAGE_PATH = "../input/plant-pathology-2020-fgvc7/images/"
TEST_PATH = "../input/plant-pathology-2020-fgvc7/test.csv"
TRAIN_PATH = "../input/plant-pathology-2020-fgvc7/train.csv"
SUB_PATH = "../input/plant-pathology-2020-fgvc7/sample_submission.csv"
MODEL_PATH = "models/plant_pathology_model.h5"

sub = pd.read_csv(SUB_PATH)
df_test = pd.read_csv(TEST_PATH)
df_train = pd.read_csv(TRAIN_PATH)

EPOCHS = 50

# Define size of the image to train on
# Remember: large image size will probably lead to higher performance
# at the expense of long training time and large memory use
IMAGE_X = 200
IMAGE_Y = 200

labels = ['healthy', 'multiple_diseases', 'rust', 'scab']

# Explore the data

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print('Training set size:', len(df_train))
for label in labels:
    print(f"\t{label}: {df_train[df_train[label]==1].shape[0]}")
print('Test set size:', len(df_test))

## check if image only belong to one class

In [ ]:
# Sum all of the labels together
df_train['number_of_classes'] = df_train['healthy'] + \
                                df_train['multiple_diseases'] + \
                                df_train['rust'] + df_train['scab']

# mean should be 1, std should be 0
df_train['number_of_classes'].mean(), df_train['number_of_classes'].std()

In [ ]:
# plot some images
nrows, ncols = 4, 4

fig, axs = plt.subplots(nrows, ncols, figsize=(16,10))
axs = axs.ravel() # make 1D array for easy plotting in for loop

for i in range(nrows*ncols):
    # show an image
    img = cv2.imread(f"{IMAGE_PATH}/{df_train['image_id'][i]}.jpg")

    # reverse BGR to RGB of opencv imported image
    axs[i].imshow(img[:,:,::-1])
    axs[i].axis(False)
    label = df_train.loc[:, 'healthy':].iloc[i, :].idxmax()
    axs[i].set_title('{} : {}'.format(df_train['image_id'][i], label))

In [ ]:
image = cv2.imread(f"{IMAGE_PATH}/{df_train['image_id'][3]}.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
fig = px.imshow(image)
fig.show()

In [ ]:
# Code thanks to https://www.kaggle.com/tarunpaparaju/plant-pathology-2020-eda-models
def load_image(image_id):
    file_path = image_id + ".jpg"
    image = cv2.imread(IMAGE_PATH + file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

train_images = df_train["image_id"][:100].apply(load_image)

red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

In [ ]:
fig = ff.create_distplot([red_values, green_values, blue_values],
                         group_labels=["R", "G", "B"],
                         colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution of channel values")
fig

## Data Generators and Augmentation

In [ ]:
#fig, axs = plt.subplots(1, 2, figsize=(16,10))
#axs[0].imshow(cv2.imread(train_paths[0]))
#axs[0].set_title('BGR')
#axs[1].imshow(cv2.imread(train_paths[0])[:,:,::-1])
#axs[1].set_title('RBG')

In [ ]:
# Convert the names of the images into a correct path
def format_path(st):
    return os.path.join(IMAGE_PATH, st + '.jpg')

# Genereate train and test paths
train_paths = df_train.image_id.apply(format_path).values
test_paths = df_test.image_id.apply(format_path).values

# Convert the labels to floats
train_labels = np.float32(df_train.loc[:,'healthy':'scab'].values)

# Split the data into validation and training sets
train_paths, valid_paths, train_labels, valid_labels = train_test_split(train_paths, train_labels, test_size=0.2, random_state=2020)

In [ ]:
train_paths.shape, valid_paths.shape, train_labels.shape, valid_labels.shape

In [ ]:
img = cv2.imread(train_paths[0])
img_max = img.max()

In [ ]:
def decode_image(filename, label=None, image_size=(IMAGE_X, IMAGE_Y), img_max = 255):
    """
    Loads, normalizes and resizes the image
    """
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    """
    Define your data augmentations here
    # only flip left/right up/down
    """
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

# Create datasets
# step 2: create a dataset returning slices of `filenames`
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
# step 3: apply every image in the dataset using `map`
train_dataset = train_dataset.map(decode_image)
train_dataset = train_dataset.map(data_augment) #only on training: helps model to become more robust
train_dataset = train_dataset.batch(BATCH_SIZE)
# step 4: shuffle random
train_dataset = train_dataset.shuffle(500) # shuffle only impact on learning, random order of classes instead of in order
# step 5: prefetch min time step during training
train_dataset = train_dataset.prefetch(AUTO)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_paths, valid_labels))
valid_dataset = valid_dataset.map(decode_image)
valid_dataset = valid_dataset.batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_paths))
test_dataset = test_dataset.map(decode_image)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [ ]:
train_dataset

### Model Definition

## try to find best fit hp

In [ ]:
# Maker sure that keras-tuner is installed
#!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import keras


def create_dense_block(inputs, n_depth, n_width):
    """ Create Dense block."""
    x = layers.BatchNormalization()(inputs)
    x = layers.Conv2D(n_width, (3,3), activation='relu', padding='same', kernel_initializer='he_normal', use_bias=False)(x)
    x = layers.Dropout(0.2)(x)
    x_list = [inputs]
    for _ in range(n_depth-1):
        x_list.append(x)
        x = layers.Concatenate()(x_list)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(n_width, (3,3), activation='relu', padding='same', kernel_initializer='he_normal', use_bias=False)(x)
        x = layers.Dropout(0.2)(x)
            
    return x


def model_builder(hp):
    # A hyperparameter search space needs to be defined. This is done by defining
    # a number of variables whose value can change over some predefined value range.
    # A variable can be an Int, Float, Boolean or Choice.
    # tune number of feature maps, dense layer size, dropout rate and learning rate
    hp_n_width = hp.Int('n_width', min_value = 16, max_value = 32, step = 8)
    hp_n_depth = hp.Int('n_depth', min_value = 2, max_value = 4, step = 1)
    hp_dense_units = hp.Int('dense_units', min_value = 128, max_value = 1024, step = 128)
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    inputs = keras.Input(shape= (IMAGE_X, IMAGE_Y, 3))
#     print('Model with n_width:{}, n_depth: {}, dense_units: {}, learning_rate: {}'.format(
#     hp_n_width, hp_n_depth, hp_dense_units,hp_learning_rate
#     ),)
    x = create_dense_block(inputs, hp_n_depth, hp_n_width)
    x = layers.Conv2D(hp_n_width, (1,1), activation='linear', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = create_dense_block(x, hp_n_depth, 2*hp_n_width)
    x = layers.Conv2D(2*hp_n_width, (1,1), activation='linear', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = create_dense_block(x, hp_n_depth, 4*hp_n_width)
    x = layers.Conv2D(4*hp_n_width, (1,1), activation='linear', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    n_dense = hp_dense_units
    x = layers.Dropout(0.2)(x)    
    x = layers.Flatten()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(n_dense, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(4, activation='softmax')(x)

    model = keras.Model(inputs, outputs)

    # define model optimization method
    model.compile(optimizer=optimizers.Adam(lr=hp_learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['categorical_accuracy'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(model_builder,
                        objective = 'val_categorical_accuracy', 
                        seed=42,
                        max_trials=5,
                        executions_per_trial=2,
                        directory='random_search20',
                        project_name='leafdisease_hp_2')

In [ ]:
# This callback is going to clear the output of the cell when searching
import IPython
class ClearTrainingOutput(tf.keras.callbacks.Callback):
      def on_train_end(*args, **kwargs):
            IPython.display.clear_output(wait = True)

In [ ]:
# Perform hyper-parameter search
steps = int(df_train.shape[0] // BATCH_SIZE)

# Pass the same paramters as in model.fit()
tuner.search(
    x=train_dataset,
    steps_per_epoch=steps,
    validation_data= valid_dataset,
    epochs=15,
    callbacks = [ClearTrainingOutput()],
    verbose=2)

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

best_hps.get('n_width'), best_hps.get('n_depth'), best_hps.get('dense_units'), best_hps.get('learning_rate')

In [ ]:
# create model with best hyperparameters
model = tuner.hypermodel.build(best_hps)

# hp = kt.HyperParameters()
# hp.Fixed('n_width', value=24)
# hp.Fixed('n_depth', value=2)
# hp.Fixed('dense_units', value=1024)
# hp.Fixed('learning_rate', value=0.0001)
# model = tuner.hypermodel.build(hp)

## training with callbacks

In [ ]:
best_w = tf.keras.callbacks.ModelCheckpoint(
    'model_best.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    period=1
)

last_w = tf.keras.callbacks.ModelCheckpoint(
    'model_last.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    mode='auto',
    period=1
)
callbacks = [best_w, last_w]

In [ ]:
steps = int(df_train.shape[0] // BATCH_SIZE)

# model.fit() now supports generators as input!
history = model.fit(
    x=train_dataset,
    steps_per_epoch=steps,
    validation_data= valid_dataset,
    epochs=100, 
    verbose=2,
    callbacks=callbacks
)

In [ ]:
print('Final Training Accuracy: {:.4f}'.format(history.history['categorical_accuracy'][-1]))
print('Final Validation Accuracy: {:.4f}'.format(history.history['val_categorical_accuracy'][-1]))
print('Final Training Loss: {:.4f}'.format(history.history['loss'][-1]))
print('Final Validation Loss: {:.4f}'.format(history.history['val_loss'][-1]))

## model definition

In [ ]:
from tensorflow.keras.applications import MobileNet, DenseNet121
from tensorflow.keras import layers

# Define model architecture
model = ... #---YOUR CODE HERE---

# compile the model
adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

### Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define checkpointing callback
mcp = ... #---YOUR CODE HERE---

# Define learnning rate schedule
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    #define your learning rate schedule
    ... #---YOUR CODE HERE---

# Create a learning rate schedule as keras callback
lrfn = build_lrfn()
lr_schedule = ... #---YOUR CODE HERE---

In [ ]:
# Visualize the learning rate across epochs
epochs_dummy = list(range(0, 50))
y = [lrfn(e) for e in epochs_dummy]
fig = go.Figure(go.Scatter(x=epochs_dummy, y=y, mode='lines+markers'))
fig.update_layout(
    yaxis = dict(
        showexponent='all',
        exponentformat='e'
    ),
    title='Learning rate schedule'
)

### Training

In [ ]:
# setup the callbacks
callbacks = ... #---YOUR CODE HERE---

# train your model
history = model.fit(... #---YOUR CODE HERE---

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def visualize_training_process(history):
    """ 
    Visualize loss and accuracy from training history
    
    :param history: A Keras History object
    """
    history_df = pd.DataFrame(history.history)
    epochs = np.arange(1, len(history_df) + 1)
    fig = make_subplots(2, 1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['categorical_accuracy'], mode='lines+markers', name='Accuracy Train'), row=1, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_categorical_accuracy'], mode='lines+markers', name='Accuracy Val'), row=1, col=1)
    
    fig.append_trace(go.Scatter(x=epochs, y=history_df['loss'], mode='lines+markers', name='Loss Train'), row=2, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_loss'], mode='lines+markers', name='Loss Val'), row=2, col=1)
    
    fig.update_layout( xaxis_title="Epochs", template="plotly_white")
    
    return fig
visualize_training_process(history)

In [ ]:
# Evaluate performance of model by plotting confusion matrix
from sklearn.metrics import confusion_matrix

# see http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools

def accuracy(y, y_pred):
    return np.sum(y == y_pred)/len(y)

def plot_confusion_matrix(cm, labels=None, title='Confusion Matrix'):
    import plotly.figure_factory as ff

    x = labels
    y = x

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in cm]

    # set up figure 
    fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='YlGnBu', showscale=True)

    # add title
    fig.update_layout(title_text=title,
                      #xaxis = dict(title='x'),
                      #yaxis = dict(title='x')
                     )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=100, l=200), width=700, height=600)
    fig.show()
    
# predict labels from validation set
y_pred = model.predict(valid_dataset)
# convert data to label number
y_pred = np.argmax(y_pred, axis=1) 
y_true = np.argmax(valid_labels, axis=1) 

# compute the confusion matrix
cm = confusion_matrix(y_true, y_pred) 

plot_confusion_matrix(cm, labels, title='Confusion_matrix Validation Set (acc={:.3f})'.format(accuracy(y_true, y_pred)))

# Save predictions

In [ ]:
# Predict labels on the test set
predictions = model.predict(test_dataset)

# Prepare the submission file
sub.loc[:, 'healthy':] = predictions
sub.to_csv('submission_densenet.csv', index=False)
sub.head()

The last step is to submit your predictions!